In [1]:
import pickle 
import pandas as pd 

In [2]:
with open('../data/data_v2.pickle','rb') as f:
    buildings = pickle.load(f)

In [3]:
print(buildings[0:5])

[defaultdict(None, {'address': '993+union+avenue+bronx', 'amenities': [], 'type': 'C1', 'details': ['21 units', '5 stories', 'Built in 1905'], 'BIN': 2004845.0}), defaultdict(None, {'address': '90-26+171+street+queens', 'amenities': [], 'type': 'D1', 'details': ['68 units', '7 stories', '4 buildings', 'Built in 2013'], 'BIN': 4209764.0}), defaultdict(None, {'address': '1017+hegeman+avenue+brooklyn', 'amenities': ['NYC Storm Zone 3'], 'type': 'S4', 'details': ['4 units', '3 stories', 'Built in 1930', 'Community District 305', 'City Council District 42', 'Police Precinct 75'], 'BIN': 3098418.0}), defaultdict(None, {'address': '1794+anthony+avenue+bronx', 'amenities': [], 'type': 'D1', 'details': ['37 units', '5 stories', 'Built in 2002'], 'BIN': 2115634.0}), defaultdict(None, {'address': '279+lee+avenue+brooklyn', 'amenities': ['Live-in Super', 'Virtual Doorman', 'NYC Storm Zone 3'], 'type': 'D1', 'details': ['10 units', '7 stories', 'Built in 2004', 'Community District 301', 'City Counc

In [4]:
building = buildings[4]


In [12]:
print(building.keys())

dict_keys(['address', 'amenities', 'type', 'details', 'apt_list', 'apts', 'BIN'])


In [3]:
apartments = pd.DataFrame()
for b in buildings: 
    if b and 'apts' in b.keys():
        apt_list = b['apts']
        amenities = b['amenities']
        i = 1
        for a in apt_list: 
            price_df = a['price_history'][0]
            price_df['address'] = b['address']
            price_df['BIN'] = b['BIN']
            room, bed, bath = None, None, None
            for deats in a['details']:
                if 'room' in deats:
                    room = deats
                elif ('studio' in deats) or ('bed' in deats): 
                    bed = deats
                elif 'bath' in deats: 
                    bath = deats 
                    
            price_df['rooms'] = room
            price_df['bedrooms'] = bed
            price_df['bath'] = bath
            price_df['apt_num'] = i
            price_df['amenities'] = str(amenities)
            apartments = apartments.append(price_df)
            i += 1
            
        

In [4]:
building['apts']

NameError: name 'building' is not defined

In [7]:
apartments.columns

Index([0, 1, 2, 'address', 'BIN', 'rooms', 'bedrooms', 'bath', 'apt_num',
       'amenities'],
      dtype='object')

In [4]:
apartments.rename(columns = {0:'date', 1:'descr', 2:'rent_price'}, inplace = True)

In [47]:
apartments.head()

,date,descr,rent_price,address,BIN,rooms,bedrooms,bath,apt_num,amenities
0,10/02/2020,Off market temporarily,"$2,750",279+lee+avenue+brooklyn,3061407.0,4 rooms,3 beds,1 bath,1,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
1,10/01/2020,Listing entered contract,"$2,750",279+lee+avenue+brooklyn,3061407.0,4 rooms,3 beds,1 bath,1,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
2,09/25/2020,Listed by StayBK LLC,"$2,750",279+lee+avenue+brooklyn,3061407.0,4 rooms,3 beds,1 bath,1,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
0,04/23/2017,Listing rented,"$2,199",279+lee+avenue+brooklyn,3061407.0,4 rooms,2 beds,1 bath,2,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."
1,03/07/2017,Listing entered contract,"$2,199",279+lee+avenue+brooklyn,3061407.0,4 rooms,2 beds,1 bath,2,"['Live-in Super', 'Virtual Doorman', 'NYC Stor..."


In [5]:
apartments['date'] = pd.to_datetime(apartments['date'])

In [6]:
apartments['year'] = apartments['date'].apply(lambda x: x.year)

In [7]:
apartments_year_subset = apartments[((apartments['year'] >= 2016.0)&(apartments['year'] <= 2017.0))| (apartments['year'] == 2019.0)]

In [8]:
apartments_grouped = apartments_year_subset.groupby(['BIN','apt_num']).filter(lambda x: x['year'].min() < 2018 and x['year'].max() >= 2019)

In [9]:
apartments_grouped

,date,descr,rent_price,address,BIN,rooms,bedrooms,bath,apt_num,amenities,year
0,2019-06-24,"Delisted by StayBK LLC. Last priced at $1,899","$1,899",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2019.0
1,2019-06-07,Later Listed by StayBK LLC,"$1,999",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2019.0
7,2016-10-02,Next Door Realty NYC Listing rented,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0
8,2016-08-05,Previously Listed by Next Door Realty NYC,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0
1,2019-10-28,Price decreased by 1%,"$2,179 ↓",4469+broadway+manhattan,1064467.0,3 rooms,1 bed,1 bath,4,"['Gym', 'Laundry in Building', 'Live-in Super'...",2019.0
...,...,...,...,...,...,...,...,...,...,...,...
9,2017-12-04,Previously Listed by ArchRock LLC,"$2,795",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0
10,2017-11-28,ArchRock LLC Listing is no longer available on...,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0
11,2017-11-07,Previously Listed by ArchRock LLC,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0
12,2016-10-16,Next Door Realty NYC Listing is no longer avai...,"$2,500",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2016.0


In [52]:
len(apartments_grouped['BIN'].unique())

123

In [53]:
len((apartments_grouped['BIN']+apartments_grouped['apt_num']).unique())

262

In [10]:
housing_df = pd.read_csv('../data/test_set.csv')

In [11]:
housing_df.columns

Index(['RegistrationID', 'BuildingID', 'BoroID', 'Boro', 'HouseNumber',
       'LowHouseNumber', 'HighHouseNumber', 'StreetName', 'StreetCode', 'Zip',
       'Block', 'Lot', 'BIN', 'CommunityBoard', 'LastRegistrationDate',
       'RegistrationEndDate', 'ADDRESS_ID', 'H_NO', 'the_geom', 'HNO_SUFFIX',
       'HYPHEN_TYP', 'SIDE_OF_ST', 'SPECIAL_CO', 'BOROCODE', 'ZIPCODE',
       'CREATED', 'MODIFIED', 'ST_NAME', 'HN_RNG_SUF', 'HN_RNG', 'PHYSICALID',
       'PRE_MODIFI', 'PRE_DIRECT', 'PRE_TYPE', 'POST_TYPE', 'POST_DIREC',
       'POST_MODIF', 'FULL_STREE', 'geometry', '250m_to_2018_dev',
       'no_proximity_newdev', 'lat', 'lon'],
      dtype='object')

In [12]:
housing_df

,RegistrationID,BuildingID,BoroID,Boro,HouseNumber,LowHouseNumber,HighHouseNumber,StreetName,StreetCode,Zip,...,PRE_TYPE,POST_TYPE,POST_DIREC,POST_MODIF,FULL_STREE,geometry,250m_to_2018_dev,no_proximity_newdev,lat,lon
0,225688,898489,2,BRONX,1743,1743,1745,DAVIDSON AVENUE,23520,10453.0,...,NaN,AVE,NaN,NaN,DAVIDSON AVE,POINT (-73.91319211844298 40.84837018078486),True,False,-73.913192,40.848370
1,360476,354310,3,BROOKLYN,2578,2578,2578,PITKIN AVENUE,70930,11208.0,...,NaN,ST,NaN,NaN,MILFORD ST,POINT (-73.87693709893625 40.67447780179333),True,False,-73.876937,40.674478
2,341209,216247,3,BROOKLYN,909,909,909,BUSHWICK AVENUE,23930,11221.0,...,NaN,AVE,NaN,NaN,BUSHWICK AVE,POINT (-73.92455362966729 40.69335103843348),True,False,-73.924554,40.693351
3,119035,26177,1,MANHATTAN,76,76,76,ORCHARD STREET,27590,10002.0,...,NaN,ST,NaN,NaN,ORCHARD ST,POINT (-73.99033507924125 40.71770738191128),True,False,-73.990335,40.717707
4,369220,339320,3,BROOKLYN,99,99,99,MONROE STREET,61730,11216.0,...,NaN,ST,NaN,NaN,MONROE ST,POINT (-73.95496530056866 40.68508748059449),True,False,-73.954965,40.685087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,351132,263486,3,BROOKLYN,143,143,143,EAST 42 STREET,36730,11203.0,...,NaN,ST,NaN,NaN,E 42 ST,POINT (-73.9380395532623 40.65234068363606),False,True,-73.938040,40.652341
1996,217785,46850,2,BRONX,2932,2932,2932,BAISLEY AVENUE,10120,10461.0,...,NaN,AVE,NaN,NaN,BAISLEY AVE,POINT (-73.82858106234812 40.83580481032219),False,True,-73.828581,40.835805
1997,142028,3331,1,MANHATTAN,832,832,832,6 AVENUE,10510,10001.0,...,AVE,NaN,NaN,NaN,AVE OF THE AMERICAS,POINT (-73.98996627725042 40.74649046908136),False,True,-73.989966,40.746490
1998,408417,810655,4,QUEENS,111-45,111-45,111-45,76 AVENUE,15840,11375.0,...,NaN,BLVD,NaN,NaN,QUEENS BLVD,POINT (-73.83639625540019 40.71767440016517),False,True,-73.836396,40.717674


In [13]:
apart_join = apartments_grouped.merge(housing_df[['BIN', 'no_proximity_newdev', 'BoroID','Boro']], on = ['BIN'])

In [21]:
apart_join

,date,descr,rent_price,address,BIN,rooms,bedrooms,bath,apt_num,amenities,year,no_proximity_newdev,BoroID,Boro,rent_price_cleaned
0,2019-06-24,"Delisted by StayBK LLC. Last priced at $1,899","$1,899",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2019.0,True,3,BROOKLYN,1899
1,2019-06-07,Later Listed by StayBK LLC,"$1,999",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2019.0,True,3,BROOKLYN,1999
2,2016-10-02,Next Door Realty NYC Listing rented,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0,True,3,BROOKLYN,1900
3,2016-08-05,Previously Listed by Next Door Realty NYC,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,"['Live-in Super', 'Virtual Doorman', 'NYC Stor...",2016.0,True,3,BROOKLYN,1900
4,2019-10-28,Price decreased by 1%,"$2,179 ↓",4469+broadway+manhattan,1064467.0,3 rooms,1 bed,1 bath,4,"['Gym', 'Laundry in Building', 'Live-in Super'...",2019.0,True,1,MANHATTAN,2179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2017-12-04,Previously Listed by ArchRock LLC,"$2,795",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False,1,MANHATTAN,2795
1570,2017-11-28,ArchRock LLC Listing is no longer available on...,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False,1,MANHATTAN,2799
1571,2017-11-07,Previously Listed by ArchRock LLC,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2017.0,False,1,MANHATTAN,2799
1572,2016-10-16,Next Door Realty NYC Listing is no longer avai...,"$2,500",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,['NYC Storm Zone 5'],2016.0,False,1,MANHATTAN,2500


In [36]:
apart_join.groupby(['BIN','apt_num'])["no_proximity_newdev"]

In [19]:
apart_join["rent_price_cleaned"] = apart_join["rent_price"].apply(lambda x: int(x[1:6].replace(",","")))

In [20]:
apart_join.to_csv("../data/apartment_data_compiled.csv")


In [26]:
#Get price change between pre 2018 and post 2018

In [39]:
perc_diff_col = apart_join.groupby(['BIN','apt_num']).apply(lambda df: (((df[df['year'] <=  2017.0]['rent_price_cleaned']).mean()) - ((df[df['year'] ==  2019.0]['rent_price_cleaned']).mean()))/((df[df['year'] <=  2017.0]['rent_price_cleaned']).mean()))

In [41]:
perc_diff = pd.DataFrame(perc_diff_col).rename(columns = {0:'price_diff'})

In [42]:
perc_diff

price_diff
BIN       apt_num            
1003107.0 2         -0.009600
          7         -0.009600
1004303.0 2          0.096386
1004471.0 1         -0.091445
          2         -0.091445
...                       ...
4206527.0 1         -0.081967
4430914.0 1          0.022403
          4          0.066667
4431832.0 3          0.021277
          6          0.028169

[262 rows x 1 columns]

In [56]:
apart_join[['BIN', 'rooms', 'bedrooms','bath', 'apt_num', 'amenities', 'no_proximity_newdev', 'BoroID',
       'Boro']].drop_duplicates().groupby(['BIN','apt_num']).filter(lambda df: len(df)>1)

,BIN,rooms,bedrooms,bath,apt_num,amenities,no_proximity_newdev,BoroID,Boro
126,3154052.0,3 rooms,studio,1 bath,3,[],True,3,BROOKLYN
130,3154052.0,2 rooms,studio,1 bath,3,[],True,3,BROOKLYN
1088,3080174.0,5 rooms,3 beds,1 bath,4,[],False,3,BROOKLYN
1108,3080174.0,4 rooms,3 beds,1 bath,7,[],False,3,BROOKLYN
1142,3080174.0,4 rooms,3 beds,1 bath,4,[],False,3,BROOKLYN
1162,3080174.0,5 rooms,3 beds,1 bath,7,[],False,3,BROOKLYN


In [57]:
apartments_cleaned = apart_join[['BIN', 'rooms', 'bedrooms','bath', 'apt_num', 'amenities', 'no_proximity_newdev', 'BoroID',
       'Boro']].drop_duplicates().drop([130,1108,1142])

In [91]:
aparts_price_change = perc_diff.merge(apartments_cleaned, on = ['BIN','apt_num'])

In [73]:
aparts_price_change
import re 

,BIN,apt_num,price_diff,rooms,bedrooms,bath,amenities,no_proximity_newdev,BoroID,Boro
0,1003107.0,2,-0.009600,4,2 beds,1 bath,[],True,1,MANHATTAN
1,1003107.0,7,-0.009600,4,2 beds,1 bath,[],True,1,MANHATTAN
2,1004303.0,2,0.096386,4,1 bed,1 bath,['NYC Storm Zone 6'],False,1,MANHATTAN
3,1004471.0,1,-0.091445,2,studio,1 bath,['NYC Storm Zone 2'],False,1,MANHATTAN
4,1004471.0,2,-0.091445,2,studio,1 bath,['NYC Storm Zone 2'],False,1,MANHATTAN
...,...,...,...,...,...,...,...,...,...,...
257,4206527.0,1,-0.081967,3,1 bed,1 bath,['Live-in Super'],True,4,QUEENS
258,4430914.0,1,0.022403,4,2 beds,1 bath,[],True,4,QUEENS
259,4430914.0,4,0.066667,6,3 beds,2 baths,[],True,4,QUEENS
260,4431832.0,3,0.021277,4,1 bed,1 bath,[],True,4,QUEENS


In [95]:
aparts_price_change['rooms'] = aparts_price_change.rooms.str.extract('(^\d*)')

In [92]:
aparts_price_change['bath'] = aparts_price_change['bath'].apply(lambda x: re.findall('\d*\.?\d+',x)[0])

In [96]:
aparts_price_change['bedrooms'] = aparts_price_change['bedrooms'].apply(lambda x: 0 if 'studio' in x else  re.findall('\d*\.?\d+',x)[0] )

In [97]:
aparts_price_change

,BIN,apt_num,price_diff,rooms,bedrooms,bath,amenities,no_proximity_newdev,BoroID,Boro
0,1003107.0,2,-0.009600,4,2,1,[],True,1,MANHATTAN
1,1003107.0,7,-0.009600,4,2,1,[],True,1,MANHATTAN
2,1004303.0,2,0.096386,4,1,1,['NYC Storm Zone 6'],False,1,MANHATTAN
3,1004471.0,1,-0.091445,2,0,1,['NYC Storm Zone 2'],False,1,MANHATTAN
4,1004471.0,2,-0.091445,2,0,1,['NYC Storm Zone 2'],False,1,MANHATTAN
...,...,...,...,...,...,...,...,...,...,...
257,4206527.0,1,-0.081967,3,1,1,['Live-in Super'],True,4,QUEENS
258,4430914.0,1,0.022403,4,2,1,[],True,4,QUEENS
259,4430914.0,4,0.066667,6,3,2,[],True,4,QUEENS
260,4431832.0,3,0.021277,4,1,1,[],True,4,QUEENS


In [101]:
aparts_price_change = aparts_price_change.drop(columns = ['amenities','Boro'])

In [102]:
aparts_price_change.to_csv('apartments_price_change.csv')